<a href="https://colab.research.google.com/github/nathan-grajan/cgan/blob/main/myCgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
from torchvision import datasets, transforms
import numpy as np
import torch.nn.utils
import torch.nn as nn
import torch.optim as optim
from torchvision.utils import save_image, make_grid
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt
from torch.autograd import Variable
print(device)



cuda


In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)), transforms.Resize(64)
                              ])

# Download and load the training data
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True)

# Download and load the test data
testset = datasets.MNIST('./data', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True)

dataiter = iter(trainloader)
images, labels = next(dataiter)

print(images.shape)

100%|██████████| 9912422/9912422 [00:00<00:00, 144095932.61it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 104879388.59it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 43222700.38it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4763823.15it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([100, 1, 64, 64])


In [ ]:
# IMPLEMENTATION 
# class Generator(nn.Module):
#     def __init__(self, latent_dim = 100, n_classes = 10, img_shape = 28):
#         super(Generator, self).__init__()

#         self.label_emb = nn.Embedding(n_classes, n_classes)
#         self.img_shape=img_shape
#         def block(in_feat, out_feat, normalize=True):
#             layers = [nn.Linear(in_feat, out_feat)]
#             if normalize:
#                 layers.append(nn.BatchNorm1d(out_feat, 0.8))
#             layers.append(nn.LeakyReLU(0.2, inplace=True))
#             return layers

#         self.model = nn.Sequential(
#             *block(latent_dim + n_classes, 128, normalize=False),
#             *block(128, 256),
#             *block(256, 512),
#             *block(512, 1024),
#             nn.Linear(1024, self.img_shape**2),
#             nn.Tanh()
#         )

#     def forward(self, z, y):
#         y = y.int()
#         # Concatenate label embedding and image to produce input
#         gen_input = torch.cat((self.label_emb(y).view(100,-1), z), -1)
#         img = self.model(gen_input)
#         img = img.view(img.size(0), 1, self.img_shape, self.img_shape)
#         return img

In [ ]:
# # IMPLEMENTATION


# class Discriminator(nn.Module):
#     def __init__(self, n_classes=10, img_shape=28):
#         super(Discriminator, self).__init__()

#         self.label_embedding = nn.Embedding(n_classes, n_classes)

#         self.model = nn.Sequential(
#             nn.Linear(n_classes + 28**2, 512),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 512),
#             nn.Dropout(0.4),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 512),
#             nn.Dropout(0.4),
#             nn.LeakyReLU(0.2, inplace=True),
#             nn.Linear(512, 1),
#         )

#     def forward(self, img, labels):
#         labels = labels.int()
#         # Concatenate label embedding and image to produce input
#         d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels).view(100,-1)), -1)
#         validity = self.model(d_in)
#         return validity

In [ ]:
# DCGAN ATTEMPT 

class Generator(nn.Module):
    def __init__(self, dim_z=100, img_size=64, num_channels=1, num_classes=10, embedding_dim = 10):
        super(Generator, self).__init__()
        self.dim_z = dim_z
        self.img_size = img_size
        self.num_channels = num_channels
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        self.label_embedding = nn.Embedding(self.num_classes, embedding_dim=self.embedding_dim)

        self.model = nn.Sequential(
            # INPUT channel x 2 x 2
            nn.ConvTranspose2d(self.dim_z + self.embedding_dim, self.img_size * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(self.img_size * 8),
            nn.ReLU(True),
            # state size. (channel*8) x 4 x 4
            nn.ConvTranspose2d(self.img_size * 8, self.img_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.img_size * 4),
            nn.ReLU(True),
            # state size. (channel*4) x 8 x 8
            nn.ConvTranspose2d(self.img_size * 4, self.img_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.img_size * 2),
            nn.ReLU(True),
            # state size. (channel*2) x 16 x 16
            nn.ConvTranspose2d(self.img_size * 2, self.img_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.img_size),
            nn.ReLU(True),
            # state size. (channel) x 32 x 32
        

            nn.ConvTranspose2d(self.img_size, self.num_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            
            # state size. (nc) x 64 x 64
        )

    def forward(self, z, labels):
        z = z.reshape(-1, self.dim_z, 1, 1)  # MAKE 4D FOR transpose (b_size X dim_z X 1 X 1)
        labels = labels.int() 
        label_emb = self.label_embedding(labels).reshape(labels.shape[0], self.embedding_dim, 1, 1) #  100 x 10 X 1 X 1
        concat = torch.cat((label_emb, z), 1)
        out = self.model(concat).view(-1, self.num_channels, self.img_size, self.img_size)

        return out  # should be num_channels x img_size * img_size

In [ ]:
# DCGAN ATTEMPT


class Discriminator(nn.Module):
    def __init__(self, z_dim=100, num_channels=1, img_size=64, num_classes=10, embedding_dim=10):
        super(Discriminator, self).__init__()

        self.z_dim = z_dim
        self.num_channels = num_channels
        self.img_size = img_size
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim


        self.label_transform = nn.Sequential(
            nn.Embedding(self.num_classes, self.embedding_dim),
            nn.Linear(self.embedding_dim, 1 * 64 * 64) # change it to 100 X 64**2
        )
        self.model = nn.Sequential(
            # NEED TO DO SPECTRAL NORM AFTER CONVOLUTIONS
            # input is (num_channels + 1) x 64 x 64
            nn.Conv2d(self.num_channels + 1, self.img_size, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            # state size. (img_size) x 32 x 32
            nn.Conv2d(self.img_size, self.img_size * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.img_size * 2),
            nn.LeakyReLU(0.2, inplace=True),

            # state size. (img_size*2 (128)) x 16 x 16
            nn.Conv2d(self.img_size * 2, self.img_size * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.img_size * 4),
            nn.LeakyReLU(0.2, inplace=True),

            # state size. (img_size*4 (256)) x 8 x 8
            nn.Conv2d(self.img_size * 4, self.img_size * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.img_size * 8),
            nn.LeakyReLU(0.2, inplace=True),
        



            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(self.img_size * 8, 1, 5, 2, 1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        # img should be input B * 1 * img_size * img_size
        labels = labels.int() 
        label_emb = self.label_transform(labels).reshape(labels.shape[0], self.num_channels, self.img_size, self.img_size) #  b_size X 1 X 64 X 64

        concat = torch.cat((label_emb, img), 1)
        out = self.model(concat)

        return out.squeeze() # should be a scalar, may need to squeeze



In [ ]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise

    batch_size, latent_dim = 100, 100
    with torch.no_grad():
      z = torch.rand(batch_size, latent_dim).to(device)
      # Get labels ranging from 0 to n_classes for n rows
      labels = np.array([num for _ in range(n_row) for num in range(n_row)])
      labels = Variable(torch.LongTensor(labels)).to(device)
      gen_imgs = generator(z, labels)

      save_image(gen_imgs.data, f"/content/images{batches_done}.png", nrow=n_row, normalize=True)





In [ ]:
# custom weights initialization called on ``netG`` and ``netD``
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# TRAINING LOOP


generator = Generator().to(device).apply(weights_init)
discriminator = Discriminator().to(device).apply(weights_init)
# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5


optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas = (beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas = (beta1, 0.999))
# lr_scheduler_G = torch.optim.lr_scheduler.ExponentialLR(optimizer_G, gamma=1.00004)
# lr_scheduler_D = torch.optim.lr_scheduler.ExponentialLR(optimizer_D, gamma=1.00004) # decays every epoch

loss = nn.BCELoss()



batch_size = 100
decay = 1.00004
dropout_probability = 0.5
num_epochs = 50
latent_dim = 100
num_classes = 10 # MNIST

# ALL LABEL VARIABLES NEED TO BE (BATCHSIZE X 1)
generator.train()
discriminator.train()
for epoch in range(num_epochs):
    for i, (data_images, data_labels) in enumerate(trainloader, 0):
        data_labels = data_labels.reshape(-1,1).float()
        real_labels = torch.ones((batch_size), dtype=torch.float32).to(device) # 100 X 1 label for each sample in batch
        fake_labels = torch.zeros((batch_size), dtype=torch.float32).to(device)
        data_images = data_images.to(device)
        data_labels = data_labels.to(device)


        ############################
        # (1) Update G network
        ###########################
        # First train generator because we need fake images for discriminator
        optimizer_G.zero_grad()
        generator.zero_grad()
        noise = torch.rand(batch_size, latent_dim).to(device)
        rand_labels = torch.randint(low=0, high=num_classes, size=(batch_size,), dtype=torch.float32).to(device)
        # scale the noise vector to the unit hypercube
        #noise = 2 * noise - 1
        gen_images = generator(noise, rand_labels)

        validity = discriminator(gen_images, rand_labels)
        gen_loss = loss(validity, real_labels)
        gen_loss.backward()
        optimizer_G.step()
        

        ############################
        # (2) Update D network
        ###########################



        optimizer_D.zero_grad()
        discriminator.zero_grad()

        validity_real = discriminator(data_images, data_labels)
        d_real_loss = loss(validity_real, real_labels)

        validity_fake = discriminator(gen_images.detach(), rand_labels)
        d_fake_loss = loss(validity_fake, fake_labels)

        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, num_epochs, i, len(trainloader), d_loss.item(), gen_loss.item())
        )

        batches_done = epoch * len(trainloader) + i
        if batches_done % 500 == 0:
            sample_image(n_row=10, batches_done=batches_done)


In [ ]:
#TESTING LOOP
generator.eval() # set generator to evaluation mode
num_samples = 1000 # number of samples to generate
batch_size = 100 # batch size for generating samples


#create a fixed noise vector for generating samples
fixed_noise = torch.randn(batch_size, latent_dim).to(device)
fixed_labels = torch.randint(low=0, high=num_classes, size=(batch_size,), dtype=torch.float32).to(device)

#generate samples
generated_samples = []
with torch.no_grad():
  for i in range(num_samples // batch_size):
    noise = fixed_noise
    labels = fixed_labels
    fake_images = generator(noise, labels)
    generated_samples.append(fake_images.cpu().detach())

    generated_samples = torch.cat(generated_samples, dim=0)

   
    sample_image(n_row=10, batches_done=0, images=generated_samples) # use the same sample_image function from the training loop